In [1]:
from hawk.keygen import hawkkeygen_unpacked as hku
from ntrugen import karamul, gen_poly
import numpy as np
from falcon import Falcon
from fft import add, mul

In [2]:
class PolyMatrix2x2:
    """
    2x2 matrix of polynomials in Z[x]/(x^n + 1)
    Determinant must be 1.
    Singleton: only one instance allowed.
    """

    def __init__(self, a, b, c, d):
        """
        Matrix:
            [ a  b ]
            [ c  d ]
        Each entry is a polynomial (list of ints).
        """
        self.a = a
        self.b = b
        self.c = c
        self.d = d

    # ---------------------------------------------------------
    # Determinant
    # ---------------------------------------------------------
    def det(self):
        """
        det = a*d - b*c  (mod x^n + 1)
        """
        ad = karamul(self.a, self.d)
        bc = karamul(self.b, self.c)
        return [ad[i] - bc[i] for i in range(len(ad))]

    def _det_is_one(self):
        det = self.det()
        expected = [1] + [0] * (len(det) - 1)
        return det == expected

    # ---------------------------------------------------------
    # Inverse (since det = 1)
    # ---------------------------------------------------------
    def inverse(self):
        """
        Since det = 1:

            M^{-1} =
            [  d  -b ]
            [ -c   a ]
        """
        minus_b = [-x for x in self.b]
        minus_c = [-x for x in self.c]

        return PolyMatrix2x2(self.d, minus_b, minus_c, self.a)

    # ---------------------------------------------------------
    # Matrix multiplication (uses karamul)
    # ---------------------------------------------------------
    def __matmul__(self, other):
        """
        Matrix multiplication using karamul.
        """
        if not isinstance(other, PolyMatrix2x2):
            raise TypeError("Can only multiply with another PolyMatrix2x2")

        a = [karamul(self.a, other.a)[i] + karamul(self.b, other.c)[i]
             for i in range(len(self.a))]

        b = [karamul(self.a, other.b)[i] + karamul(self.b, other.d)[i]
             for i in range(len(self.a))]

        c = [karamul(self.c, other.a)[i] + karamul(self.d, other.c)[i]
             for i in range(len(self.a))]

        d = [karamul(self.c, other.b)[i] + karamul(self.d, other.d)[i]
             for i in range(len(self.a))]

        return PolyMatrix2x2(a, b, c, d)

    # ---------------------------------------------------------
    # Pretty polynomial printer
    # ---------------------------------------------------------
    @staticmethod
    def _poly_to_str(p):
        terms = []
        for i, coef in enumerate(p):
            if coef == 0:
                continue
            if i == 0:
                terms.append(str(coef))
            elif i == 1:
                terms.append(f"{coef}x")
            else:
                terms.append(f"{coef}x^{i}")
        return " + ".join(terms) if terms else "0"

    # ---------------------------------------------------------
    # Representation
    # ---------------------------------------------------------
    def __repr__(self):
        return (
            "[ "
            + self._poly_to_str(self.a) + "    "
            + self._poly_to_str(self.b) + " ]\n"
            "[ "
            + self._poly_to_str(self.c) + "    "
            + self._poly_to_str(self.d) + " ]"
        )

In [3]:
def rand_poly_2x2_gen():
    n1 = [1] + 255 * [0]
    n2, n3 = gen_poly(256), gen_poly(256)
    temp = karamul(n2, n3)
    temp[0] += 1
    n4 = temp
    return PolyMatrix2x2(n1, n2, n3, n4)

In [4]:
f, g, F, G = hku(8)[:4]

In [5]:
B = PolyMatrix2x2(f, g, F, G)

In [6]:
U1 = rand_poly_2x2_gen()
xw = B.__matmul__(U1)

In [7]:
assert xw._det_is_one() == True

In [8]:
U2 = rand_poly_2x2_gen()
zw = B.__matmul__(U2)

In [9]:
assert zw._det_is_one() == True

In [10]:
zw_inv = zw.inverse()

### Rounding params
$$p_1 = (20109 · (2 · 256)) + 1$$
$$q = 2^{32}$$
$$p = 2^5$$

In [11]:
p1 = 20109 * 2 * 256 + 1
q = 1 << 32
p = 1 << 5

In [13]:
falcon = Falcon(256)

In [14]:
sk, vk = falcon.keygen()

In [15]:
ig = falcon.sign(sk, b'testing falcon sign')

In [16]:
falcon.verify(vk, b'testing falcon sign', ig)

True

### Generating xtoken

In [17]:
xtoken = xw.__matmul__(zw)

In [18]:
def lwr(lst, q, p):
    """
    Apply LWR rounding from modulus q to modulus p.
    
    For each a in lst:
        a <- a mod q
        return round(p * a / q) mod p
    """
    result = []
    for a in lst:
        a_mod = a % q
        # print(a_mod)
        scaled = (a_mod * p) / q
        rounded = int(round(scaled)) % p
        # print(scaled, rounded)
        result.append(rounded)
        # return
    return result

In [19]:
a, b, c, d = lwr(xtoken.a, q, p1), lwr(xtoken.b, q, p1), lwr(xtoken.c, q, p1), lwr(xtoken.d, q, p1)
xtoken_r = PolyMatrix2x2(a, b, c, d)

In [20]:
rsign = falcon.raw_sign(sk, b'testing falcon sign') ## this gives a vector of 2 polynomials

In [21]:
temp_poly = PolyMatrix2x2(rsign[0], 256 * [0], rsign[1], 256 * [0])

In [22]:
xtag = xw.__matmul__(temp_poly)

### Generating xtag

In [23]:
a, b, c, d = lwr(xtag.a, q, p), lwr(xtag.b, q, p), lwr(xtag.c, q, p), lwr(xtag.d, q, p)
xtag_r = PolyMatrix2x2(a, b, c, d)

In [24]:
ytag = zw_inv.__matmul__(temp_poly)

### Generating ytag

In [25]:
a, b, c, d = lwr(ytag.a, q, p1), lwr(ytag.b, q, p1), lwr(ytag.c, q, p1), lwr(ytag.d, q, p1)
ytag_r = PolyMatrix2x2(a, b, c, d)

### Generating xtoken * ytag

In [26]:
hypoth = xtoken_r.__matmul__(ytag_r)

In [27]:
a, b, c, d = lwr(hypoth.a, p1, p), lwr(hypoth.b, p1, p), lwr(hypoth.c, p1, p), lwr(hypoth.d, p1, p)
hypoth_r = PolyMatrix2x2(a, b, c, d)

In [28]:
xtag_r

[ 1x^67 + 1x^92 + 31x^174 + 1x^241    0 ]
[ 1 + 1x^2 + 31x^4 + 31x^7 + 31x^9 + 31x^10 + 31x^12 + 31x^15 + 2x^16 + 1x^17 + 1x^19 + 1x^20 + 31x^21 + 1x^22 + 30x^29 + 31x^32 + 1x^34 + 1x^37 + 1x^39 + 1x^40 + 1x^42 + 31x^43 + 31x^44 + 31x^47 + 31x^50 + 1x^56 + 1x^57 + 31x^58 + 1x^63 + 31x^66 + 1x^68 + 1x^71 + 31x^72 + 1x^73 + 1x^74 + 1x^75 + 1x^77 + 31x^78 + 31x^84 + 31x^86 + 31x^87 + 1x^91 + 1x^97 + 31x^98 + 1x^99 + 31x^101 + 31x^104 + 31x^106 + 1x^111 + 1x^114 + 31x^115 + 1x^116 + 1x^117 + 1x^118 + 31x^120 + 31x^121 + 31x^122 + 31x^123 + 31x^124 + 31x^130 + 31x^131 + 1x^133 + 1x^134 + 1x^136 + 31x^138 + 31x^139 + 1x^140 + 31x^141 + 1x^142 + 1x^145 + 31x^146 + 1x^148 + 31x^149 + 1x^150 + 1x^151 + 1x^153 + 2x^154 + 31x^157 + 31x^158 + 31x^159 + 31x^161 + 1x^162 + 31x^163 + 1x^168 + 1x^171 + 31x^175 + 31x^178 + 30x^181 + 31x^184 + 1x^186 + 2x^188 + 1x^189 + 1x^190 + 1x^191 + 31x^192 + 31x^195 + 31x^198 + 31x^199 + 31x^200 + 31x^201 + 31x^202 + 31x^203 + 2x^205 + 1x^208 + 31x^209 + 1x^214 + 

In [29]:
hypoth_r

[ 28 + 5x + 22x^2 + 17x^3 + 21x^4 + 4x^5 + 10x^6 + 10x^7 + 24x^8 + 29x^9 + 13x^10 + 31x^11 + 2x^12 + 29x^13 + 9x^14 + 2x^15 + 31x^16 + 8x^17 + 21x^18 + 12x^19 + 22x^20 + 17x^21 + 7x^22 + 8x^23 + 6x^24 + 2x^26 + 17x^27 + 5x^28 + 8x^29 + 25x^30 + 19x^31 + 25x^32 + 25x^33 + 24x^35 + 6x^36 + 31x^37 + 26x^38 + 3x^39 + 5x^40 + 30x^41 + 27x^42 + 24x^43 + 27x^44 + 26x^45 + 18x^46 + 27x^47 + 29x^48 + 7x^49 + 30x^51 + 29x^52 + 16x^53 + 20x^54 + 18x^55 + 27x^56 + 9x^58 + 31x^59 + 4x^60 + 19x^61 + 10x^62 + 19x^63 + 19x^64 + 1x^65 + 12x^66 + 8x^67 + 4x^68 + 25x^69 + 5x^70 + 26x^71 + 31x^72 + 2x^73 + 5x^74 + 15x^75 + 28x^76 + 29x^77 + 31x^78 + 25x^79 + 29x^80 + 24x^81 + 15x^82 + 13x^84 + 30x^85 + 30x^86 + 25x^87 + 12x^88 + 16x^89 + 6x^90 + 12x^91 + 27x^92 + 9x^93 + 12x^94 + 29x^95 + 6x^96 + 26x^97 + 15x^98 + 2x^100 + 10x^102 + 1x^103 + 12x^104 + 13x^105 + 3x^106 + 24x^107 + 29x^108 + 6x^109 + 27x^110 + 12x^111 + 30x^112 + 22x^113 + 23x^114 + 19x^115 + 18x^116 + 5x^117 + 13x^118 + 1x^119 + 3x^120 + 2